In [2]:
!pip install --upgrade google-cloud-aiplatform

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 28.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.31.0
    Uninstalling google-cloud-aiplatform-1.31.0:
      Successfully uninstalled google-cloud-aiplatform-1.31.0


In [57]:
from datetime import datetime
import llm_evals
import logging
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

In [58]:
LOGGING_OUTPUT=["stdout"] # stdout, csv, jsonl, sqlite(TODO)
METRICS=["exact_match"] # exact_match, rouge, levenshtein, cosine(TODO), custom(TODO)
MODEL=["gemini-pro"]
#  **Google:**
# 	- gemini-pro	
# 	- text-bison@001
# 	- text-bison@002
# 	- text-unicorn@001
# TODO: 
# - text-bison-32k@002
# 	- MedLM-large
# 	- MedLM-medium
# 	- code-bison-32k@002
# 	- code-bison@001
# 	- code-gecko@001
# 	- code-gecko@002
#  **OpenAI:**
# 	- GPT-3.5 Turbo
# 	- GPT-4
# 	- GPT-4 Turbo

configs = {
    "model": MODEL,
    "metrics": METRICS,
    "logging_output": LOGGING_OUTPUT,
    "prompt_file": "sample2.jsonl"
}

In [59]:
PROJECT_ID = "eazy-evals"
LOCATION = "us-central1" #e.g. us-central1
vertexai.init()

In [60]:
def generate(configs, prompt="tell me a joke"):
    #TODO: wrap in an iterator for the models listed in the configs array
    model = GenerativeModel(configs["model"][0])
    responses = model.generate_content(
        [prompt],
        generation_config={
            "max_output_tokens": 2048,
            "temperature": 0.2,
            "top_p": 1
        },
        stream=True,
        )
    model_output = ""
    for response in responses:
        #TODO: needs different response parsers depending on the model selection
        model_output += (response.candidates[0].content.parts[0].text)
    return model_output

In [62]:
generate(configs,"what is five plus four?")

'9'

In [63]:
# Example loading method
from urllib.parse import urlparse
import json

def load_examples(uri):
    """takes a local or remote URI for input/output ground truth examples in .jsonl format. 
    returns a list of json objects with the input/output pairs."""
    
    uri_parts = urlparse(uri)
    # with open (uri_parts.path, 'r') as f: 
    #     prompt_list.append(json.loads(f))
    with open(uri_parts.path, 'r') as json_file:
        prompt_list = [json.loads(line) for line in json_file]

    return prompt_list

In [64]:
# examples = load_examples('sample2.jsonl')

In [65]:
# for key in examples:
    # generate(configs,key["input_text"])

In [75]:
def evaluate_metrics(ground_truth, generated_text, eval_function=exact_match):
    metric = eval_function[0]
    if metric == "exact_match":
        eval_score=exact_match(ground_truth,generated_text)
    elif metric == "levenshtein":
        eval_score=levenshtein(ground_truth,generated_text)
    elif metric == "rouge":
        eval_score=rouge(ground_truth,generated_text)
    else: eval_score = "not specified"
    return eval_score

In [76]:
# def run_evals(configs):
#     prompt_list = load_examples(configs["prompt_file"])
#     if "csv" in configs["logging_output"]:
#         output_filename = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".csv"
#         with open(output_filename, "w") as f:
#             f.write("input_text, output_text, generated_text\n")
            
#     for key in prompt_list: 
#         generated_text = generate(configs, key["input_text"])

#         for log_method in configs["logging_output"]:
            
#             match log_method:
#                 case "stdout":
#                     print(generated_text)
#                 case "csv":
#                     with open(output_filename, "a") as f:
#                         f.write(key["input_text"] + ",")
#                         f.write(key["output_text"] + ",")
#                         f.write(generated_text + "\n")
#                 case _:
#                     print(generated_text)
def run_evals(configs):
    prompt_list = load_examples(configs["prompt_file"])
    output_filename = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    if "csv" in configs["logging_output"]:
        with open(output_filename + ".csv", "w") as f:
            f.write("input_text, output_text, generated_text, score\n")
            
    for line in prompt_list: 
        input_text = line["input_text"]
        ground_truth = line["output_text"]
        generated_text = generate(configs, line["input_text"])
        eval_score = evaluate_metrics(ground_truth,generated_text,configs["metrics"])
        for log_method in configs["logging_output"]:
            if log_method == "stdout":
                print(input_text,generated_text,eval_score)
            elif log_method == "csv":
                with open(output_filename + ".csv" , "a") as f:  # Changed to "a" for appending
                    f.write(input_text + ",")
                    f.write(ground_truth + ",")
                    f.write(generated_text + ",")
                    f.write(eval_score + "\n")
            else:
                print(input_text,generated_text,eval_score) 
    # evaluate_metrics

In [77]:
run_evals(configs)

question: How many people live in Beijing? context: With over 21 million residents, Beijing is the world's most populous national capital city and is China's second largest city after Shanghai. It is located in Northern China, and is governed as a municipality under the direct administration of the State Council with 16 urban, suburban, and rural districts.[14] Beijing is mostly surrounded by Hebei Province with the exception of neighboring Tianjin to the southeast; together, the three divisions form the Jingjinji megalopolis and the national capital region of China. 21 million 0
question: How many parishes are there in Louisiana? context: The U.S. state of Louisiana is divided into 64 parishes (French: paroisses) in the same manner that 48 other states of the United States are divided into counties, and Alaska is divided into boroughs. 64 1
